## Constants

In [1]:
import re
import string;
from nltk.corpus import stopwords

EMOJI_PATTERN=re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002500-\U00002BEF"  # chinese char
                           u"\U00002702-\U000027B0"
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           u"\U0001f926-\U0001f937"
                           u"\U00010000-\U0010ffff"
                           u"\u2640-\u2642"
                           u"\u2600-\u2B55"
                           u"\u200d"
                           u"\u23cf"
                           u"\u23e9"
                           u"\u231a"
                           u"\ufe0f"  # dingbats
                           u"\u3030"
                           "]+", re.UNICODE);
PUNCT_TO_REMOVE = string.punctuation.replace("'", "")
STOP_WORDS=stopwords.words("english")
STOP_WORDS.extend(["'s","'m","'ve","s","#"])

TRAIN_SPLIT = 0.6
TEST_SPLIT = 0.2
LEARNING_RATE = 0.1
EPOCHS= 10

## Import libraries and datasets

In [6]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import re
import spacy
import en_core_web_sm

from nltk.tokenize import word_tokenize, RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer

from bs4 import BeautifulSoup
from html import unescape

reddit_suidiceWatch = pd.read_csv('reddit_depression_suicidewatch.csv') #depression and SuicideWatch. long texts (limit it somehow). row=20365
# depressive_tweets = pd.read_csv('depressive_tweets_processed.csv', sep = '|', header = None, usecols = range(0,9), nrows = 3200) #all depressed, no label needed. row= 4078
tweet_mental_health_classification = pd.read_csv('tweet-mental-health-classification-train.csv') # stressed, anxious, ..... first column, no column name. row= 1.048.575
training160000 = pd.read_csv('training.1600000.processed.noemoticon.csv',encoding='latin-1') # 0=negative, 2=notr, 4=positive. first column, no column name. row= 1.048.575

training160000.drop(columns=training160000.columns[1:5], axis=1, inplace=True)
column_names=["labels",'tweets']
training160000.columns = column_names
new_cols = ["tweets","labels"]
training160000=training160000.reindex(columns=new_cols)

# depressive_tweets.drop(columns=depressive_tweets.columns[-3:], axis=1, inplace=True)
# depressive_tweets.drop(columns=depressive_tweets.columns[0:5], axis=1, inplace=True)

# column_names_d=['tweet']
# depressive_tweets.columns = column_names_d

nlp = en_core_web_sm.load()

## Inspect and Split Datasets

### 1.training160000 (positive, negative)

In [7]:
positive_training160000 = training160000[training160000['labels'] == 4]
DF_POSITIVE = positive_training160000
print("positive size:")
DF_POSITIVE['labels'] = "Positive"
print(len(DF_POSITIVE))
DF_POSITIVE.head(3)

positive size:
800000


,tweets,labels
799999,I LOVE @Health4UandPets u guys r the best!!,Positive
800000,im meeting up with one of my besties tonight! ...,Positive
800001,"@DaRealSunisaKim Thanks for the Twitter add, S...",Positive


### 2.reddit_suidiceWatch (Depression, Suidical)

In [8]:
def divideRedditTextToTweetLength(paragraph):
    lines = []
    line = ''
    if(len(line) + len(paragraph) + 1 < 240):
        return [paragraph]
    for sentence in (s.strip()+'.' for s in paragraph.split('.')[:-1]):
        if len(line) + len(sentence) + 1 >= 240: #overfitting.
            lines.append(line)
            line = sentence
        else:                                   
            line += ' ' + sentence  
    print(lines)
    return lines

In [10]:
# no time:( suicidal_reddit_list = [divideRedditTextToTweetLength(paragraph) for paragraph in suicidal_reddit['tweets'].tolist()]

suicidal_reddit = reddit_suidiceWatch[reddit_suidiceWatch['label'] == 'SuicideWatch']
DF_SUICIDAL = suicidal_reddit
DF_SUICIDAL['label'] = "Suicidal"
DF_SUICIDAL.rename(columns = {'text':'tweets','label':'labels'}, inplace = True)
DF_SUICIDAL['tweets'] = DF_SUICIDAL['tweets'].str.slice(0,240) # paragrafı parçalara ayırmak yerine ilk 240ı almayı tercih ettik (şimdilik)

print("depressive_reddit_suicide size:")
print(len(DF_SUICIDAL))
DF_SUICIDAL.head(3)

depressive_reddit_suicide size:
9992


,tweets,labels
3,I am so exhausted of this. Just when I think I...,Suicidal
5,I am 20 year old with some good friends but I ...,Suicidal
8,it is looming around the corner again. It alwa...,Suicidal


In [11]:
depressive_reddit = reddit_suidiceWatch[reddit_suidiceWatch['label'] == 'depression']
print("depressive_reddit_depression size:")
depressive_reddit.rename(columns = {'text':'tweets','label':'labels'}, inplace = True)
depressive_reddit['labels'] = "Depressive"
print(len(depressive_reddit))
depressive_reddit

depressive_reddit_depression size:
10371


,tweets,labels
0,I recently went through a breakup and she said...,Depressive
1,"I do not know how to navigate these feelings, ...",Depressive
2,"So I have been with my bf for 5 months , and h...",Depressive
4,I have been severly bullied since i was 5 till...,Depressive
6,My mom made me go to a camp that she knows I h...,Depressive
...,...,...
20355,cannot even decide where to start. Low self es...,Depressive
20359,that is what has happened to me last week. And...,Depressive
20360,Ever just feel alone in a house full of people...,Depressive
20361,Politicians. Neighbors. Corporations. Society....,Depressive


In [12]:
type(suicidal_reddit['tweets'])


pandas.core.series.Series

### 3.depressive_tweets

In [14]:
print(len(depressive_tweets))
# print(depressive_tweets)
# print(depressive_reddit)

depressive_tweets['label']='Depressive'
depressive_tweets.rename(columns = {'tweet':'tweets','label':'labels'}, inplace = True)
depressive_tweets

NameError: name 'depressive_tweets' is not defined

#### merge depressive_tweets and depressive_reddit


In [15]:
# DF_DEPRESSIVE=pd.concat([depressive_reddit,depressive_tweets],axis=0)
DF_DEPRESSIVE=depressive_reddit
DF_DEPRESSIVE['tweets'] = DF_DEPRESSIVE['tweets'].str.slice(0,240)
print(len(DF_DEPRESSIVE))
DF_DEPRESSIVE.head(3)

10371


,tweets,labels
0,I recently went through a breakup and she said...,Depressive
1,"I do not know how to navigate these feelings, ...",Depressive
2,"So I have been with my bf for 5 months , and h...",Depressive


### tweet_mental_health_classification (Stressed, Anxious, Normal, Lonely)

In [16]:
DF_STRESSED = tweet_mental_health_classification[tweet_mental_health_classification['labels'] == 'Stressed']
print(len(DF_STRESSED))
DF_STRESSED['tweets'].head(5)

6840


0     sending solidarity whoever doctor manage incre...
11                            feel beautiful woman sad 
18    tire fight foreign occupier fight defend homel...
21    boy get slaughter helpless mother could scream...
27    sad thing disinformation truth come damage alr...
Name: tweets, dtype: object

In [17]:
DF_ANXIOUS = tweet_mental_health_classification[tweet_mental_health_classification['labels'] == 'Anxious']
print(len(DF_ANXIOUS))
DF_ANXIOUS.head(5)

8388


,tweets,labels
1,need see hair amp beard gat book appointment b...,Anxious
6,good point remember 1013 leave alone pokie clu...,Anxious
8,okay ik lot people go want im make gc ashnikko...,Anxious
14,ron desantis danger florida democracy want pre...,Anxious
19,fact get mecha work boy girl doggy style posit...,Anxious


In [18]:
DF_NORMAL = tweet_mental_health_classification[tweet_mental_health_classification['labels'] == 'Normal']
print(len(DF_NORMAL))
DF_NORMAL.head(5)

7973


,tweets,labels
2,next time meet someone new dont ask ask love,Normal
4,raise hand junhoes ocean lotion life rent free...,Normal
5,mariposa de barrio teach matter guy forever ch...,Normal
7,mori wip white dress,Normal
9,im gonna say,Normal


In [19]:
DF_LONELY = tweet_mental_health_classification[tweet_mental_health_classification['labels'] == 'Lonely']
print(len(DF_LONELY))
DF_LONELY.head(5)

6788


,tweets,labels
3,surprise someone love give la senza gift box r...,Lonely
10,people suffer know difficult mourn one person ...,Lonely
15,2021 im bring back tune 20142016 era aka actua...,Lonely
16,base abuja get 5years experience office admini...,Lonely
20,guess need feel comfort biden protect us evil ...,Lonely


## Create corpus

In [20]:
CORPUS=pd.concat([DF_LONELY,DF_NORMAL,DF_ANXIOUS,DF_STRESSED,DF_DEPRESSIVE,DF_SUICIDAL,DF_POSITIVE],axis=0)
CORPUS.labels.value_counts()

Positive      800000
Depressive     10371
Suicidal        9992
Anxious         8388
Normal          7973
Stressed        6840
Lonely          6788
Name: labels, dtype: int64

Görüldüğü üzere oranlar çok dengesiz. Dengelenmeye ihtiyacı var

## Preprocessing and Balancing

In [21]:
def preprocess(tweet):
#     print(tweet)
    ans = tweet[1].lower() # lowercase
    ans = tweet.lower() # lowercase
    ans = re.sub('@[^\s]+', '', ans)   # remove mentions
    soup = BeautifulSoup(unescape(ans), 'lxml')
    ans = soup.text
#     print("after -----------")
    ans = EMOJI_PATTERN.sub(r'', ans)  #remove emojis TODO: remove also symbols like :) :D vs.
    ans = re.sub(r'http\S+', '', ans)  # remove links
    ans = " ".join(ans.split())        # remove unnecessarily spaces
    
    data = nlp(ans)
    filtered_tokens=[]
    for token in data:
        if token.is_stop or token.is_punct:
            continue;
        filtered_tokens.append(token.lemma_) #lemmatizes token

    ans = " ".join(filtered_tokens) 
    
#     print(ans)
    return ans

In [22]:
DF_LONELY_processed = DF_LONELY
DF_LONELY_processed['tweets'] = [preprocess(text) for text in DF_LONELY['tweets']]

In [23]:
DF_NORMAL_processed = DF_NORMAL
DF_NORMAL_processed['tweets'] = [preprocess(text) for text in DF_NORMAL['tweets']]

In [24]:
DF_ANXIOUS_processed = DF_ANXIOUS
DF_ANXIOUS_processed['tweets'] = [preprocess(text) for text in DF_ANXIOUS['tweets']]

In [25]:
DF_STRESSED_processed = DF_STRESSED
DF_STRESSED_processed['tweets'] = [preprocess(text) for text in DF_STRESSED['tweets']]
DF_STRESSED_processed

,tweets,labels
0,send solidarity doctor manage incredibly distr...,Stressed
11,feel beautiful woman sad,Stressed
18,tire fight foreign occupi fight defend homelan...,Stressed
21,boy slaughter helpless mother scream father tr...,Stressed
27,sad thing disinformation truth come damage peo...,Stressed
...,...,...
29970,tension,Stressed
29975,faang tire maybe tina tact,Stressed
29977,todays democrat party party hollywood celebrit...,Stressed
29983,sad newscloris insanely talented laugh cry dro...,Stressed


In [26]:
DF_DEPRESSIVE_processed = DF_DEPRESSIVE
DF_DEPRESSIVE_processed['tweets'] = [preprocess(text) for text in DF_DEPRESSIVE['tweets']]
DF_DEPRESSIVE_processed

C:\Users\gokz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


,tweets,labels
0,recently go breakup say want friend say try ta...,Depressive
1,know navigate feeling new feeling stretch unde...,Depressive
2,bf 5 month tell depressed week particular happ...,Depressive
4,severly bully 5 till 15 result depressed misan...,Depressive
6,mom camp know hate hate day good time midnight...,Depressive
...,...,...
20355,decide start low self esteem personality know ...,Depressive
20359,happen week week depression insomnia sleep goo...,Depressive
20360,feel house people room cry loneliness suck,Depressive
20361,politician neighbor corporation society cultur...,Depressive


In [27]:
DF_SUICIDAL_processed = DF_SUICIDAL
DF_SUICIDAL_processed['tweets'] = [preprocess(text) for text in DF_SUICIDAL['tweets']]
DF_SUICIDAL_processed

C:\Users\gokz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


,tweets,labels
3,exhausted think finally rest think maybe thing...,Suicidal
5,20 year old good friend tired problem bullying...,Suicidal
8,loom corner come like wave like moon hear sile...,Suicidal
9,foodand thing judge have weird view maybe delu...,Suicidal
13,zoloft focalin change life past year well week...,Suicidal
...,...,...
20353,sorry sorry study joke suffer consequence go f...,Suicidal
20354,tell end life person love indication long time...,Suicidal
20356,week disgusting thing cheat husband good frien...,Suicidal
20357,way head way change life well happy feel great...,Suicidal


In [28]:
default_size=60000

DF_POSITIVE_under = DF_POSITIVE.sample(default_size)
DF_DEPRESSIVE_over = DF_DEPRESSIVE_processed.sample(default_size, replace=True)
DF_SUICIDAL_over = DF_SUICIDAL_processed.sample(default_size, replace=True)
DF_ANXIOUS_over = DF_ANXIOUS_processed.sample(default_size, replace=True)
DF_NORMAL_over = DF_NORMAL_processed.sample(default_size, replace=True)
DF_STRESSED_over = DF_STRESSED_processed.sample(default_size, replace=True)
DF_LONELY_over = DF_LONELY_processed.sample(default_size, replace=True)

In [30]:
DF_POSITIVE_processed = DF_POSITIVE_under
DF_POSITIVE_processed['tweets'] = [preprocess(text) for text in DF_POSITIVE_under['tweets']]
DF_POSITIVE_processed

KeyboardInterrupt: 

In [31]:
CORPUS_BALANCED=pd.concat([
    DF_POSITIVE_processed,
    DF_DEPRESSIVE_over,
    DF_SUICIDAL_over,
    DF_ANXIOUS_over,
    DF_NORMAL_over,
    DF_STRESSED_over,
    DF_LONELY_over]
    ,axis=0)

print('Random under-sampling:')
print(CORPUS_BALANCED.labels.value_counts())

Random under-sampling:
Positive      60000
Depressive    60000
Suicidal      60000
Anxious       60000
Normal        60000
Stressed      60000
Lonely        60000
Name: labels, dtype: int64


In [32]:
CORPUS_BALANCED.to_csv('CORPUS_BALANCED.csv')

## Split test and train data

In [33]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test=train_test_split(
    CORPUS_BALANCED.tweets,
    CORPUS_BALANCED.labels,
    test_size=0.2,
    random_state=2022,
    stratify=CORPUS_BALANCED.labels)

In [34]:
X_train,Y_train

(15686                                                  chive
 14234                                         ring need ring
 1302543    @mileycyrus hey miley its my birthday! i hope ...
 1130651    Show @hedon was amazing, it's getting better e...
 27418      m fine absence probably continue tho ve play v...
                                  ...                        
 25700                     screenshot 5 dogecoin jun 1st 2021
 17226      month panic attack therapy feel numb know soun...
 5731       begin think giant exercise incredibly gullible...
 28317      listen young man talk directly ear need favor ...
 18126      time think suicide 7 year ago 15 time act temp...
 Name: tweets, Length: 336000, dtype: object,
 15686          Normal
 14234         Anxious
 1302543      Positive
 1130651      Positive
 27418        Stressed
               ...    
 25700          Normal
 17226      Depressive
 5731         Stressed
 28317          Lonely
 18126      Depressive
 Name: labels, Leng

## Vectorize??

In [35]:
Y_train

15686          Normal
14234         Anxious
1302543      Positive
1130651      Positive
27418        Stressed
              ...    
25700          Normal
17226      Depressive
5731         Stressed
28317          Lonely
18126      Depressive
Name: labels, Length: 336000, dtype: object

In [36]:
vectorizer = CountVectorizer(ngram_range=(1, 2))

vectorizer.fit(X_train)
 
# Printing the identified Unique words along with their indices
print("Vocabulary: ", vectorizer.vocabulary_)
 
# Encode the Document
vector = vectorizer.transform(document)
 
# Summarizing the Encoded Texts
print("Encoded Document is:")
print(vector.toarray())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



NameError: name 'document' is not defined

## smote, balance the corpus

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(DF_STRESSED, DF_DEPRESSIVE)

# Train

MultinomialNB | Unigram | Processed data

In [37]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

classifier_mb1=Pipeline([
    ('vectorizer_bow',CountVectorizer()),
    ('Multi NB', MultinomialNB())
])
classifier_mb1.fit(X_train,Y_train)
Y_pred= classifier_mb1.predict(X_test)
print(classification_report(Y_test,Y_pred))


              precision    recall  f1-score   support

     Anxious       0.53      0.48      0.50     12000
  Depressive       0.72      0.79      0.75     12000
      Lonely       0.55      0.62      0.58     12000
      Normal       0.94      0.71      0.81     12000
    Positive       0.98      0.98      0.98     12000
    Stressed       0.92      0.92      0.92     12000
    Suicidal       0.72      0.80      0.76     12000

    accuracy                           0.76     84000
   macro avg       0.77      0.76      0.76     84000
weighted avg       0.77      0.76      0.76     84000



In [38]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

classifier_mb2=Pipeline([
    ('vectorizer_bow',CountVectorizer(ngram_range=(1,2))),
    ('Multi NB', MultinomialNB())
])
classifier_mb2.fit(X_train,Y_train)
Y_pred= classifier_mb2.predict(X_test)
print(classification_report(Y_test,Y_pred))

              precision    recall  f1-score   support

     Anxious       0.61      0.49      0.54     12000
  Depressive       0.95      0.96      0.96     12000
      Lonely       0.58      0.72      0.64     12000
      Normal       1.00      0.88      0.94     12000
    Positive       0.99      0.98      0.98     12000
    Stressed       0.98      0.99      0.98     12000
    Suicidal       0.93      0.98      0.96     12000

    accuracy                           0.86     84000
   macro avg       0.86      0.86      0.86     84000
weighted avg       0.86      0.86      0.86     84000



In [39]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

classifier_mb3=Pipeline([
    ('vectorizer_bow',CountVectorizer(ngram_range=(1,3))),
    ('Multi NB', MultinomialNB())
])
classifier_mb3.fit(X_train,Y_train)
Y_pred= classifier_mb3.predict(X_test)
print(classification_report(Y_test,Y_pred))

              precision    recall  f1-score   support

     Anxious       0.62      0.48      0.54     12000
  Depressive       0.96      0.99      0.97     12000
      Lonely       0.58      0.73      0.65     12000
      Normal       1.00      0.90      0.95     12000
    Positive       0.99      0.98      0.99     12000
    Stressed       0.98      0.99      0.98     12000
    Suicidal       0.96      0.99      0.98     12000

    accuracy                           0.87     84000
   macro avg       0.87      0.87      0.87     84000
weighted avg       0.87      0.87      0.87     84000



In [226]:
manual_test_list=["I want to kill myself",
                  "awesome, everything is okay",
                  "thank you so much",
                  "moving another country",
                  "i hate all of you and going to kill myself",
                  "what is the point of living?",
                  "death is everywhere",
                  "overcome depression",
                  "song is great",
                  "i saw a sad woman"]

sonuc1=classifier_mb1.predict(manual_test_list)
print(sonuc1)


['Suicidal' 'Positive' 'Positive' 'Normal' 'Suicidal' 'Suicidal'
 'Suicidal' 'Depressive' 'Positive' 'Stressed']


In [227]:
sonuc_list = [preprocess(text) for text in manual_test_list]

sonuc1=classifier_mb1.predict(sonuc_list)
print(sonuc1)


['Suicidal' 'Positive' 'Positive' 'Depressive' 'Suicidal' 'Suicidal'
 'Suicidal' 'Depressive' 'Positive' 'Stressed']


## Export modal


In [42]:
import joblib
classifier_file=open("../modal/mental_illness_detector.pkl","wb")
joblib.dump(classifier_mb2,classifier_file)
classifier_file.close()